In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.layers import Embedding, Masking, Concatenate, GRU, Dense, Reshape
from model import CNN_with_mask

In [2]:
df = pd.read_csv('test.csv')

In [3]:
# rally: 1292
# Score: 891
# Errors: 1288
# 至少其中一個一定有 -> 先用 Error 來看，沒有再用 Score 來看

In [4]:
def get_rally_result(df):
    result_label = []
    ignor_game_rally_index = []
    team_col = [c for c in df.columns if 'Team' in c]
    for _, df_rally in df.groupby(['Game', 'Rally']):
        start_from = df.iloc[df_rally.index[0]][team_col].tolist()

        result = df_rally[df_rally['Score'] == 1]
        if(len(result) != 0):
            if(result.iloc[0][team_col].tolist() == start_from):
                result_label.append([0, 1])
            else:
                result_label.append([1, 0])
        else:
            result = df_rally[df_rally['Errors'] == 1]
            if(len(result) == 0):
                ignor_game_rally_index.append((df_rally.iloc[0]['Game'], df_rally.iloc[0]['Rally']))
            elif(result.iloc[0][team_col].tolist() != start_from):
                result_label.append([0, 1])
            else:
                result_label.append([1, 0])
    return tf.constant(result_label, dtype=float), ignor_game_rally_index

In [5]:
label, ignor = get_rally_result(df)

In [6]:
def get_padding_data(df, ignor):
    space_col = [c for c in df.columns if 'Space' in c]
    action_col = [c for c in df.columns if 'Action' in c]
    result_col = ['Errors', 'Score', 'Nothing']
    others_col = [c for c in df.columns if c not in space_col and c not in action_col and c not in result_col and c != 'Game' and c != 'Rally']
    team_col = [c for c in df.columns if 'Team' in c]

    rally_set = []
    rally_space_set = []
    rally_action_set = []
    for _, df_rally in df.groupby(['Game', 'Rally']):   # each rally in one game
        if((df_rally.iloc[0]['Game'], df_rally.iloc[0]['Rally']) in ignor):
            continue
        start_from = df.iloc[df_rally.index[0]][team_col].tolist()
        shot_set = []
        shot_space_set = []
        shot_action_set = []
        
        atk_sequence = []
        atk_space_sequence = []
        atk_action_sequence = []

        curr_team = start_from
        for _, shot in df_rally.iterrows():
            if(shot[team_col].tolist() != curr_team):
                shot_set.append(atk_sequence)
                shot_space_set.append(atk_space_sequence)
                shot_action_set.append(atk_action_sequence)
                
                curr_team = shot[team_col].tolist()

                atk_sequence = []
                atk_space_sequence = []
                atk_action_sequence = []

            atk_space_sequence.append(shot[space_col])
            atk_action_sequence.append(shot[action_col])
            atk_sequence.append(shot[others_col])
        
        # the last shot
        shot_set.append(atk_sequence)
        shot_space_set.append(atk_space_sequence)
        shot_action_set.append(atk_action_sequence)

        # one rally has been finished
        shot_set = pad_sequences(shot_set, maxlen=3, padding='post')
        shot_space_set = pad_sequences(shot_space_set, maxlen=3, padding='post')
        shot_action_set = pad_sequences(shot_action_set, maxlen=3, padding='post')

        # one rally has been finished
        rally_set.append(shot_set)
        rally_space_set.append(shot_space_set)
        rally_action_set.append(shot_action_set)

    padded_rally_set = pad_sequences(rally_set, dtype=float, padding='post')
    padded_rally_space_set = pad_sequences(rally_space_set, dtype=float, padding='post')
    padded_rally_action_set = pad_sequences(rally_action_set, dtype=float, padding='post')
    
    return padded_rally_set, padded_rally_space_set, padded_rally_action_set

In [7]:
space_replace = {list(df.groupby('Space').groups.keys())[i]: i+1 for i in range(len(df.groupby('Space')))}
action_replace = {list(df.groupby('Action').groups.keys())[i]: i+1 for i in range(len(df.groupby('Action')))}

df = pd.get_dummies(df, columns=['Team', 'No.'])
df = df.replace(space_replace)
df = df.replace(action_replace)

In [8]:
rally_set, rally_space_set, rally_action_set= get_padding_data(df, ignor)

# rally數, 最大回合數in one rally, 3, feature數
print(rally_set.shape)
print(rally_space_set.shape)
print(rally_action_set.shape)
print(label.shape)

(1291, 14, 3, 36)
(1291, 14, 3, 1)
(1291, 14, 3, 1)
(1291, 2)


---

In [9]:
rally_space_set = rally_space_set.squeeze()
rally_action_set = rally_action_set.squeeze()

In [10]:
rally_set_tensor = tf.convert_to_tensor(rally_set)
rally_space_set_tensor = tf.convert_to_tensor(rally_space_set)
rally_action_set_tensor = tf.convert_to_tensor(rally_action_set)
rally_result_tensor = tf.convert_to_tensor(label)

In [11]:
# rally_num = rally_set.shape[0]
rally_size = rally_set.shape[1]
shot_size = 3
feature_dim = (rally_set.shape[-1], len(df.groupby('Space'))+1, len(df.groupby('Action'))+1)
space_embed_size = 8
action_embed_size = 8
shot_embed_size = 16

In [12]:
def create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size):
    '''
    framework: 
    1. 對 space, action 做 embeding, (input, output) = (feature_dim, embed_size)
    2. concat space, action, others 成一個 embedded vector for each atk, (input) =  ([feature_dim, embed_size, embed_size])
    3. 先做 embedding
    4. CNN, filters = shot_embed_size
    5. GRU
    '''
    # each input: 三個維度, rally shot feature
    input_others = keras.Input(shape=(rally_size, shot_size, feature_dim[0]))
    input_space = keras.Input(shape=(rally_size, shot_size))
    input_action = keras.Input(shape=(rally_size, shot_size))

    # space & action 先做 embedding, 再和 others concat
    embed_space_layer = Embedding(input_dim=feature_dim[1], output_dim=space_embed_size, mask_zero=True, name='Space_Embedding')
    embed_action_layer = Embedding(input_dim=feature_dim[2], output_dim=action_embed_size, mask_zero=True, name='Action_Embedding')
    masking_layer = Masking(mask_value=0)   # for input_others (還沒有經過mask)
    concat_layer = Concatenate(name='Input_Concat')

    embed_shot_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=3, name='Shot_Embedding')

    cnn_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=1, name='CNN_Layer')
    gru_layer = GRU(units=16, name='GRU_Layer')
    dense_layer = Dense(units=2, activation='softmax')

    # forward
    inputs = [input_others, input_space, input_action]

    embed_space = embed_space_layer(input_space)
    embed_action = embed_action_layer(input_action)
    masked_others = masking_layer(tf.cast(input_others, tf.float32))
    embed_input = concat_layer([masked_others, embed_space, embed_action])
    embed_shot = tf.squeeze(embed_shot_layer(embed_input), axis=2)

    cnn_output = cnn_layer(embed_shot)
    gru_output = gru_layer(cnn_output)
    output = dense_layer(gru_output)
    model = keras.Model(inputs=inputs, outputs=output, name='Classification')
    return model

In [13]:
model = create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size)
model.summary()

Model: "Classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 14, 3, 36)]  0                                            
__________________________________________________________________________________________________
tf.cast (TFOpLambda)            (None, 14, 3, 36)    0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 14, 3)]      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 14, 3)]      0                                            
_____________________________________________________________________________________

In [14]:
def split_data(others_tensor, space_tensor, action_tensor, label_tensor):
    l = label_tensor.shape[0]
    split_persentage = int(l*0.7)

    train_space = space_tensor[:split_persentage]
    train_action = action_tensor[:split_persentage]
    train_others = others_tensor[:split_persentage]
    train_label = label[:split_persentage]

    test_space = space_tensor[split_persentage:]
    test_action = action_tensor[split_persentage:]
    test_others = others_tensor[split_persentage:]
    test_label = label[split_persentage:]

    train_x = [train_others, train_space, train_action]
    train_y = train_label

    test_x = [test_others, test_space, test_action]
    test_y = test_label

    return train_x, train_y, test_x, test_y

In [15]:
train_x, train_y, test_x, test_y = split_data(rally_set_tensor, rally_space_set_tensor, rally_action_set_tensor, rally_result_tensor)

In [16]:
# regularizer = tf.keras.regularizers.l2(0.01)
optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
epochs = 30
# callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [17]:
model.fit(train_x, train_y, epochs=epochs)

Epoch 1/30
29/29 [==============================] - 2s 5ms/step - loss: 0.6449 - accuracy: 0.6645
Epoch 2/30
29/29 [==============================] - 0s 5ms/step - loss: 0.6398 - accuracy: 0.6656
Epoch 3/30
29/29 [==============================] - 0s 4ms/step - loss: 0.6374 - accuracy: 0.6656
Epoch 4/30
29/29 [==============================] - 0s 4ms/step - loss: 0.6356 - accuracy: 0.6656
Epoch 5/30
29/29 [==============================] - 0s 5ms/step - loss: 0.6345 - accuracy: 0.6656
Epoch 6/30
29/29 [==============================] - 0s 5ms/step - loss: 0.6214 - accuracy: 0.6733
Epoch 7/30
29/29 [==============================] - 0s 5ms/step - loss: 0.6077 - accuracy: 0.6932
Epoch 8/30
29/29 [==============================] - 0s 4ms/step - loss: 0.6041 - accuracy: 0.6800: 0s - loss: 0.6015 - accuracy: 0.68
Epoch 9/30
29/29 [==============================] - 0s 5ms/step - loss: 0.5896 - accuracy: 0.6811
Epoch 10/30
29/29 [==============================] - 0s 4ms/step - loss: 0.5782 - 

In [18]:
result = model.evaluate(test_x, test_y)

13/13 [==============================] - 0s 1ms/step - loss: 0.6700 - accuracy: 0.8067


In [19]:
y_pred = model.predict(test_x)

In [20]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [21]:
argmax_y_pred = np.argmax(y_pred, axis=1)
argmax_test_y = np.argmax(test_y, axis=1)

In [22]:
from sklearn.metrics import accuracy_score, roc_auc_score
from evaluate import calculate_BS, f1, show_eval_result_2class

acc_score = accuracy_score(argmax_test_y, argmax_y_pred)
f1_score = f1(argmax_test_y, argmax_y_pred)
auc_score = roc_auc_score(test_y, y_pred)
BS = calculate_BS(test_y, y_pred, 2)

In [23]:
show_eval_result_2class(acc_score, f1_score, auc_score, BS)

accuracy:    0.81
f1:          0.74
auc:         0.90
BS:          0.16
